# Capstone Project 1:Data Wrangling

In [ ]:
# import all packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import pytz
import collections
import shutil
import os
from glob import glob

In [ ]:
# configure DataFrame display settings
pd.options.display.max_columns = None

In [ ]:
# set the output directory for the pre-processed files
dir_outfiles = 'C:/Users/Jonathon.Poage/Desktop/Springboard/Capstone_Documentation/Project_1/Data_Files/preprocessed_data/'

# if the output directory exists, delete it
if os.path.exists(dir_outfiles):
    shutil.rmtree(dir_outfiles)

# make the output directory
os.makedirs(dir_outfiles)

### Flight Data

In [ ]:
# set the pathname to the data files
dir_flight_data = 'C:/Users/Jonathon.Poage/Desktop/Springboard/Capstone_Documentation/Project_1/Data_Files/flight_performance_data/'

# make the list of filenames
filepaths_flight_data = glob(dir_flight_data + '*.csv')

In [ ]:
# list of airport codes: top 10 busiest airports
airport_codes = ['ATL', 'LAX', 'ORD', 'DFW', 'JFK',
                 'DEN', 'SFO', 'LAS', 'SEA', 'MIA']

# list of airline codes: top 8 largest airlines
carrier_codes = ['WN', 'DL', 'AA', 'UA',
                 'B6', 'AS', 'NK', 'F9']

In [ ]:
# idendify the date column
datecol = ['FL_DATE']

# dict of timezones for airports
airport_tzs = {'ATL':'US/Eastern',
               'LAX':'US/Pacific',
               'ORD':'US/Central',
               'DFW':'US/Central',
               'JFK':'US/Eastern',
               'DEN':'US/Mountain',
               'SFO':'US/Pacific',
               'LAS':'US/Pacific',
               'SEA':'US/Pacific',
               'MIA':'US/Eastern'}

# set dtypes for the input data
dict_dtypes_flight_data = {'CANCELLED':bool, 'DIVERTED':bool,
                           'CRS_DEP_TIME':str, 'DEP_TIME':str,
                           'WHEELS_OFF':str, 'WHEELS_ON':str,
                           'CRS_ARR_TIME':str, 'ARR_TIME':str}

In [ ]:
# print the start time for the cell execution
print(pd.Timestamp.now())

# read the data into a list of daframes, and preprocess the dataframes
dfs_flight_data = []
for f in filepaths_flight_data:
    df = pd.read_csv(f,
                     dtype=dict_dtypes_flight_data)
    
    # filter the dataframe
    df = df[df.OP_UNIQUE_CARRIER.isin(carrier_codes)
          & df.ORIGIN.isin(airport_codes)
          & df.DEP_DELAY.notnull()
          & df.FL_DATE.notnull()
          & (df.CANCELLED == False)]
    
    # create column with timestamps for CRS departure times
    df['crs_dep_ts'] = pd.to_datetime(df['FL_DATE'] + ' ' + df['CRS_DEP_TIME'], errors='coerce', format='%Y-%m-%d %H%M')

    # add a column with timezone info
    df['origin_timezone'] = df.ORIGIN.map(airport_tzs)
    
    # create column with timestamps for actual departure times
    df['dep_ts'] = df['crs_dep_ts'] + pd.to_timedelta(df['DEP_DELAY'], unit='m', errors='coerce')
    
    # create column with timestamps for wheels off times
    df['wheels_off_ts'] = df['dep_ts'] + pd.to_timedelta(df['TAXI_OUT'], unit='m', errors='coerce')
    
    # filter the dataframe based on timestamp null values
    df = df[df.wheels_off_ts.notnull()
          & df.dep_ts.notnull()
          & df.crs_dep_ts.notnull()]
    
    # append the dataframe
    dfs_flight_data.append(df)

# concatenate the pre-processed dataframes
df_f = pd.concat(dfs_flight_data)

# print the end time for the cell execution
print(pd.Timestamp.now())

In [ ]:
# get rid of unwanted columns
df_f.drop(['FL_DATE',
           'OP_CARRIER_AIRLINE_ID',
           'TAIL_NUM',
           'ORIGIN_AIRPORT_ID',
           'ORIGIN_AIRPORT_SEQ_ID',
           'ORIGIN_CITY_MARKET_ID',
           'DEST_AIRPORT_ID',
           'DEST_AIRPORT_SEQ_ID', 
           'DEST_CITY_MARKET_ID',
           'CRS_DEP_TIME',
           'DEP_TIME',
           'WHEELS_OFF',
           'WHEELS_ON',
           'CRS_ARR_TIME',
           'ARR_TIME',
           'ARR_TIME_BLK',
           'Unnamed: 38'],
           axis='columns',
           inplace=True)

# reset the index
df_f.reset_index(drop=True, inplace=True)

### Weather Data

In [ ]:
# set the pathname to the data files
dir_weather_data = 'C:/Users/Jonathon.Poage/Desktop/Springboard/Capstone_Documentation/Project_1/Data_Files/weather_data/'

# make the list of filenames
filepaths_weather_data = glob(dir_weather_data + '*.txt')

In [ ]:
# identify the timestamp columns
ts_cols_weather = ['valid']

In [ ]:
# print the start time for the cell execution
print(pd.Timestamp.now())

# read the data into a list of daframes, and preprocess the dataframes
dfs_weather_data = []
for fw in filepaths_weather_data:
    dfw = pd.read_csv(fw,
                      skipinitialspace=True,
                      parse_dates=ts_cols_weather,
                      na_values='M')
    
    # filter the dataframe
    dfw = dfw[dfw.valid.notnull()
            & dfw.station.isin(airport_codes)]
    
    # filter out an anomalous data point
    dfw = dfw[~((dfw.station == 'DEN')
                & (dfw.valid == pd.to_datetime('2017-05-11 14:53:00')))]
    
    # convert wind directions of 360 to 0
    dfw.loc[(dfw.drct == 360), 'drct'] = 0
    
    # add a column with timezone info
    dfw['station_timezone'] = dfw.station.map(airport_tzs)
    
    # append the list of dataframes
    dfs_weather_data.append(dfw)
    
# concatenate the pre-processed dataframes
df_w = pd.concat(dfs_weather_data)

# print the end time for the cell execution
print(pd.Timestamp.now())

In [ ]:
# drop unecessary columns
df_w.drop(['metar'],
          axis='columns',
          inplace=True)

# reset the index
df_w.reset_index(drop=True, inplace=True)

## Merge the DataFrames

In [ ]:
# initialize a list of merged dataframes
dfs_merged = []

# loop over airport codes
for c in airport_codes:
    # slice by airport and sort by time
    left = df_f[df_f.ORIGIN == c].sort_values('crs_dep_ts')
    right = df_w[df_w.station == c].sort_values('valid')
    
    # merge left and right.  left-join, 1 hour window "backward" search.
    # NOTE: I can't use "forward" or "nearest" because my pandas version is outdated
    merged_lr = pd.merge_asof(left,
                              right,
                              left_on='crs_dep_ts',
                              right_on='valid',
                              tolerance=pd.to_timedelta('1 hours'))
    
    # filter out rows without a right side match
    merged_lr = merged_lr[merged_lr.valid.notnull()]
    
    # append the list of merged dataframes
    dfs_merged.append(merged_lr)
    
# concatenate the merged dataframes while resetting the index
df_m = pd.concat(dfs_merged,
                 ignore_index=True)

In [ ]:
# set the output filepath
name_outfile_m = 'merged_flight_weather_data_preprocessed.csv'
filepath_outfile_m = dir_outfiles + name_outfile_m

# write the concatenated dataframe to a csv file
df_m.to_csv(filepath_outfile_m, index=False)